In [30]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
import textwrap

from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
import textwrap

def text_to_pdf(input_file, output_file, font_path):
    """
    Converts a structured text file with `-----` page delimiters into a properly formatted PDF.
    Titles marked with `###` will be centered, bold, and use the correct Devanagari font.
    """
    # Register the font
    pdfmetrics.registerFont(TTFont("DevanagariFont", font_path))
    
    # Page settings
    page_width, page_height = letter
    margin = 70  # Left/right margin
    top_margin = page_height - 80  # Top margin
    bottom_margin = 50  # Bottom margin
    line_height = 20  # Line spacing
    title_spacing = 30  # Extra space after a title
    max_width = page_width - 2 * margin  # Max text width for wrapping
    
    # Create the PDF
    c = canvas.Canvas(output_file, pagesize=letter)
    c.setFont("DevanagariFont", 12)
    
    # Read the input text and split into pages based on `-----` delimiters
    with open(input_file, "r", encoding="utf-8") as f:
        content = f.read()
    pages = content.split("-------------")  # Split text into pages
    
    # Initialize page variables
    page_number = 1

    def add_page_number():
        """Adds a page number at the bottom of the current page."""
        c.setFont("DevanagariFont", 10)
        c.drawCentredString(page_width / 2, bottom_margin - 10, f"पृष्ठ {page_number}")

    # Process each "page" of text
    for page_content in pages:
        lines = page_content.strip().split("\n")  # Split the page content into lines
        y = top_margin
        first_line = True  # Track the first line of the page
        
        for line in lines:
            line = line.strip()
            if not line:
                continue  # Skip empty lines
            
            # Check if it's a title (starts with ###)
            if line.startswith("###"):
                title_text = line.replace("###", "").strip()
                c.setFont("DevanagariFont", 14)  # Larger font for title
                c.drawCentredString(page_width / 2, y, title_text)  # Centered title
                y -= title_spacing  # Extra spacing after title
                c.setFont("DevanagariFont", 12)  # Reset font for regular text
                first_line = False
                continue
            
            # Wrap text within the maximum width
            wrapped_lines = textwrap.wrap(line, width=85)  # Adjust width as needed
            for wrapped_line in wrapped_lines:
                if y < bottom_margin + line_height:  # Check if a new page is needed
                    add_page_number()
                    c.showPage()
                    c.setFont("DevanagariFont", 12)
                    y = top_margin
                    page_number += 1
                
                c.drawString(margin, y, wrapped_line)
                y -= line_height
        
        # Add a page number and start a new page for the next section
        add_page_number()
        c.showPage()
        c.setFont("DevanagariFont", 12)
        page_number += 1
    
    # Save the PDF
    c.save()
    print(f"PDF saved to {output_file}")

In [32]:
input_file = "bhajan_sangrah.md"  # Replace with the path to your input text file
output_file = "formatted_book.pdf"  # Replace with the desired output file name
font_path = "/Users/soodoku/Desktop/Noto_Sans_Devanagari/static/NotoSansDevanagari-Regular.ttf"
text_to_pdf(input_file, output_file, font_path = font_path)

PDF saved to formatted_book.pdf
